# Seminario 1: Gestión de memoria.

Python es un **lenguaje orientado a objetos**. En un primer momento esto no resulta muy aparente, ya que hasta ahora hemos utilizado un estilo de programación *imperativo*. No obstante, aún sin ser completamente conscientes de ello, realmente hemos estado creando y manejando objetos en nuestros programas. En Python absolutamente *todo* lo que se manipula son objetos: números, cadenas, listas, tuplas, diccionarios, funciones, módulos, etc.

La mayoría de los lenguajes orientados a objetos, incluyendo Python, cuentan con una gestión dinámica de memoria integrada en el mismo *runtime* del lenguaje, lo cual permite ordenar la creación y destrucción de objetos durante la ejecución de los programas.

Python, gracias a tratarse de un lenguaje interpretado, es además un **lenguaje dinámico**: así, algunos objetos se crean bajo demanda, es decir, en el mismo instante en que se necesitan, sin necesidad de crearlos explícitamente, ni de especificar su tipo/clase, que Python determina a partir del modo en que se utilizan dichos objetos. Así mismo, los objetos que quedan sin uso (independientemente de si se crearon bajo demanda o explícitamente) se destruyen automáticamente, de nuevo sin necesidad de hacerlo explícitamente.

Estas características del lenguaje Python facilitan enormemente la programación de todo tipo de aplicaciones pero, a la vez, ese dinamismo y automatismo ocultan ciertos aspectos importantes, en concreto el funcionamiento de la gestión de memoria. Esto no supone un problema para programadores con cierta experiencia, pero sí puede serlo para quienes se acaban de iniciar en la programación con Python.

Para ir más allá del nivel de iniciación, es necesario entender *qué sucede realmente* cuando se ejecutan nuestros programas; sólo así podremos tomar las mejores decisiones y emplear las técnicas más adecuadas. Ese conocimiento no implica tener que preocuparnos de todos los detalles; antes al contrario, el dominio de la herramienta y de la técnica nos permitirá emplearlas de forma óptima. Y cuando nos tropecemos (inevitablemente) con problemas aparentemente inexplicables, saber qué sucede realmente será una ayuda inestimable para solucionarlos.

## Creación y eliminación dinámica de objetos.

Comenzaremos analizando de forma práctica cómo funciona ese dinamismo de Python en lo que respecta a la creación de objetos bajo demanda, así como las condiciones en que los objetos se destruyen automáticamente. Nótese que por *creación* de un objeto entendemos que se reserva memoria para él; y por *destrucción* de un objeto entendemos que la memoria destinada a ese objeto se retorna al *pool* de memoria libre, recuperándola para otros usos. Sin embargo, en condiciones normales lo importante no es cuándo se destruye un objeto, sino cuándo se marca para que sea destruido posteriormente.

### El recolector de basura (*garbage collector*).

Muchos lenguajes que cuentan con gestión dinámica de memoria poseen un componente de su *runtime* llamado **recolector de basura** (*garbage collector*). Su función es recolectar la memoria ocupada por objetos que ya no están en uso, con el objetivo de devolverla al *pool* de memoria libre, de modo que quede disponible para otros usos (como la creación de nuevos objetos). El recolector de basura no funciona de manera continua, sino sólo cuando el intérprete de Python lo considera más conveniente.

Para eliminar automáticamente los objetos que ya no están en uso, el intérprete de Python utiliza la técnica del *recuento de referencias*. Todo objeto cuenta entre sus atributos con un *contador de referencias*, que almacena cuántas referencias existen hacia ese objeto, sea desde variables o desde otros objetos. Cada vez que se crea una referencia a un objeto, se incrementa ese contador; cada vez que se elimina una referencia, se decrementa. Y cuando ese contador llega a cero, el objeto en cuestión **se marca como huérfano** (*orphaned object*), lo que significa que será liberado posteriormente por el recolector de basura.

## Objetos referenciados desde variables.

El concepto de variable en Python es muy diferente del que se utiliza en los lenguajes de programación clásicos. En éstos, una variable identifica con su nombre un área de la memoria reservada para almacenar un dato o colección de datos (como un entero, un array, etc.); por tanto, los atributos de una variable en sentido clásico son: el tipo de datos que almacena, su ubicación en memoria, y por supuesto el contenido de dicha ubicación (el valor de la variable).

Por otro lado, en Python una variable tan sólo representa una *referencia* a un objeto (en esencia, su dirección de memoria), siendo dicha referencia el único atributo de la variable. Es decir, una variable de Python no posee más información del objeto al que referencia: ni su tipo/clase, ni su contenido; éstos son atributos del objeto referenciado.

### Python vs C

Por ejemplo, dentro de una función escrita en lenguaje C podríamos tener este código:

    int n;    
    n = 5;
    
La primera línea *declara* la variable `n` como de tipo entero, y además la *define*, reservando para ella una ubicación concreta en memoria. En la segunda línea, la asignación copia el valor del literal `5` (que, por su forma, representa el valor 5, de tipo entero) en la ubicación de `n`, por supuesto reemplazando su contenido previo (desconocido en este caso). Obsérvese que ha sido necesario declarar primero el tipo de datos que puede almacenar `n`. Téngase además en cuenta que C es un lenguaje compilado: para poder ejecutar ese programa, primero habrá de ser analizado por el compilador de C, el cual creará un programa ejecutable conteniendo tanto el área de memoria reservada para la variable `n`, como el código que copia el valor entero 5 a esa área de memoria.

En Python, escribiríamos simplemente:

    n = 5

Al ejecutar ese código, el intérprete de Python empezará por crear un nuevo objeto, reservando en ese mismo instante un área de memoria para dicho objeto e inicializando a continuación sus atributos: su tipo (`int`), su valor (5), y otros atributos, como el número de referencias que existen hacia ese nuevo objeto (inicialmente cero). A continuación realizará la asignación, asociando al identificador `n` la referencia al objeto recién creado (esencialmente, su ubicación en memoria), e incrementará en 1 el contador de referencias de dicho objeto. Obsérvese que no ha habido que declarar la variable `n`, ya que no posee información sobre el tipo de dato, ni tampoco ha sido necesario reservar memoria explícitamente.

## La función `id`.
Considerense las dos asignaciones siguientes, en las que se crean dos objetos: el número `5` y la cadena `'hola'`.

In [2]:
n = 5
s = 'hola'

La función de Python `id` retorna un número entero que representa la *identidad* de un objeto. En realidad, esa identidad no es otra cosa que la ubicación del objeto en memoria, es decir, la referencia en sí, pero expresada como un `int`. Véamoslo:

In [3]:
print(f'n({n}) -> {id(n)}')
print(f's({s}) -> {id(s)}')

n(5) -> 9752288
s(hola) -> 140151334342064


## La sentencia `del`.
La *sentencia* `del` permite eliminar una *referencia* a un objeto. En el siguiente ejemplo, primero hacemos que la variable `t` haga referencia al mismo objeto que `s` y a continuación eliminamos la referencia `s`. Es decir, hacemos desaparecer la variable `s` pero no el objeto referenciado por `s`.

In [4]:
t = s

print("Las variables s y t apuntan al mismo objeto")
print(f's({s}) -> {id(s)}')
print(f't({t}) -> {id(t)}')

print(f'\nAhora eliminamos la variable s({s}) -> {id(s)}')
del s

print(f'\nPero aún queda la variable t apuntando al objeto')
print(f't({t}) -> {id(t)}')

Las variables s y t apuntan al mismo objeto
s(hola) -> 140151334342064
t(hola) -> 140151334342064

Ahora eliminamos la variable s(hola) -> 140151334342064

Pero aún queda la variable t apuntando al objeto
t(hola) -> 140151334342064


El objeto `'hola'` sigue existiendo porque aún conserva una referencia (desde la variable `t`). Si ahora eliminamos esa referencia, el objeto habrá quedado huérfano y será procesado por el recolector de basura para devolver ese bloque de memoria al sistema.

In [5]:
print(f'Eliminamos la variable t({t}) -> {id(t)})')
del t

Eliminamos la variable t(hola) -> 140151334342064)


Nóetese que ya no podemos ver qué ha pasado con el objeto `'hola'`, porque no tenemos forma de acceder (hemos perdido todas sus referencias).

## Referencias desde unos objetos a otros.

Por supuesto, no sólo las variables pueden hacer referencia a objetos. También hay objetos que pueden hacer referencia a otros objetos, como listas, tuplas, diccionarios, conjuntos, etc. A continuación creamos una lista `l` con tres elementos; en realidad, la lista `l` apunta a un objeto que es un secuencia contigua en memoria de tres referencias a objetos. Lo que vemos al hacer un `print` de la lista son los objetos referenciados. 

In [6]:
l = [5, 'hola', [1, 2, 3]]
print(l)

[5, 'hola', [1, 2, 3]]


Ahora vamos a definir algunas variables más que harán referencia a los elementos de la lista. 

In [7]:
n = l[0]
s = l[1]
sub_l = l[2]

Si añadimos algo a la lista referenciada por la variable `sub_l`, veremos que se trata en realidad de una sola lista, referenciada también desde `l[0]`.

In [8]:
sub_l.append(8.56)
print(f'sub_l: {sub_l}')
print(f'l: {l}')

sub_l: [1, 2, 3, 8.56]
l: [5, 'hola', [1, 2, 3, 8.56]]


## La función `getrefcount` del módulo `sys`.

El módulo `sys` incluye una función `getrefcount` que retorna el número de referencias al objeto que se le pasa como parámetro. Hay que tener en cuenta un par de cosas:

1. Para obtener valores con sentido, vamos a usar la función `getrefcount` sólo con objetos mutables. La razón de ello es que Python puede utilizar técnicas de optimización de memoria para compartir objetos inmutables a gran escala, dado que los objetos inmutables pueden ser compartidos (referenciados desde varios sitios) sin riesgo. Por tanto, en los ejemplos que siguen utilizaremos `getrefcount` con listas, ya que son objetos mutables y esas técnicas de optimización no se aplican.
2. `getrefcount` retorna una referencia más de las que cabe esperar. Esto sucede porque, al pasar la dirección del objeto como parámetro, durante la ejecución de la función existe esa referencia adicional, que desaparece una vez la función `getrefcount` termina de ejecutarse.

In [9]:
from sys import getrefcount

Veamos qué nos dice `getrefcount` de la listas `l` y `sub_l`:

In [10]:
print(f'l: {getrefcount(l)}')
print(f'sub_l: {getrefcount(sub_l)}')

l: 2
sub_l: 3


Tras restar esa referencia adicional que siempre nos da `getrefcount`, vemos que hay *una* referencia al objeto referenciado por `l`, que es la propia variable `l`, y *dos* referencias al objeto referenciado por `sub_l`: la propia variable `sub_l` más la referencia correspondiente al último elemento de `l`.

Podemos eliminar tranquilamente la variable `sub_l` sin que ello afecte a `l`:

In [11]:
del sub_l
print(l)

[5, 'hola', [1, 2, 3, 8.56]]


La razón, como hemos visto, es que ese objeto sigue estando referenciado desde el último elemento de la lista `l`. 

Ahora comprobamos que ha disminuido en 1 el número de referencias al objeto `[1, 2, 3, 8.56]` tras hacer `del sub_l`:

In [12]:
print(getrefcount(l[-1]))

2


¿Y qué pasa si eliminamos la variable `l` haciendo `del l`? La cuenta de referencias de la lista caerá a 0 y pasará a ser un objeto *huérfano*, por lo que será liberado por el recolector de basura. Pero no sólo eso: los tres elementos de la lista son *referencias* al entero `5`, la cadena `hola` y la lista `[1, 2, 3, 8.56]`, respectivamente. Al eliminar la variable `l`, las cuentas de referencias de esos tres objetos disminuirán en 1. El entero `5` aún estará referenciando por la variable `n`, pero la cadena `hola` y la lista `[1, 2, 3, 8.56]` carecen de otras referencias, por lo que sus cuentas caerán a 0 y el recolector de basura acabará liberando la memoria que ocupaban.

In [13]:
del l
print(f'n({n}) -> {id(n)}')
print(f'n: {getrefcount(n)}')

n(5) -> 9752288
n: 270


Nótese la cuenta desorbitada del entero `5`, debido a que el intérprete de Python lo está reutilizando en muchos lugares (a pesar de que nosotros sólo tenemos acceso desde la vairable `n`).

Las referencias a objetos se pueden eliminar explícitamante mediante la sentencia `del`, pero también al hacer que una variable haga referencia a otro objeto. Véamoslo con un ejemplo:

In [14]:
s = 'Hello'
s = s + ', ' + 'World'
print(s)

Hello, World


¿Qué ha sucedido ahí?
1. En la primera sentencia, se crea el objeto `'Hello'` y al asignarlo a `s` se crea también una referencia a dicha cadena.
2. En la segunda sentencia, en la parte derecha de la asignación, la expresión de concatenación *crea un nuevo objeto* que contendrá la concatenación de la cadena referenciada por `s` (`'Hello'`) con las cadenas literales `', '` y `'World'` (cada una de ellas dando lugar a un nuevo objeto, de existencia efímera).
3. Por último, la asignación de la segunda sentencia reemplaza la referencia de `s` a `'Hello'` por una nueva referencia a la cadena recién creada. Como consecuencia, el objeto `'Hello'` pierde la única referencia que tenía y se convierte en un objeto huérfano, cuya memoria será devuelta al sistema por el recolector de basura.

## Referencias en el paso de parámetros a funciones.

En Python también se usan las referencias a objetos como mecanismo de paso de parámetros a funciones. En los lenguajes clásicos se emplean mecanismos como el paso por valor (se copia el valor del parámetro en la llamada al parámetro formal de la función), o el paso por referencia (sólo válido para pasar variables, se copia la dirección de la variable). El mecanismo que utiliza Python se denomina formalmente **paso de parámetros por referencia a objeto**. Véamoslo con un ejemplo sencillo:

In [15]:
def to_float(v):
    print(f'Dentro de to_float :: id(v) = {id(v)}')
    for i in range(len(v)):
        v[i] = float(v[i])

vector = [1, 7, 0, 3, 5]
print(f'Antes de la llamada :: vector = {vector}')
print(f'Antes de la llamada :: id(vector) = {id(vector)}')
to_float(vector)
print(f'Después de la llamada :: vector = {vector}')

Antes de la llamada :: vector = [1, 7, 0, 3, 5]
Antes de la llamada :: id(vector) = 140151334352832
Dentro de to_float :: id(v) = 140151334352832
Después de la llamada :: vector = [1.0, 7.0, 0.0, 3.0, 5.0]


Obsérvese que `vector` (fuera de la función) y `v` (el parámetro formal de la función) *hacen referencia al mismo objeto*. La razón es que al hacer la llamada `to_float(vector)` simplemente se ha copiado la referencia almacenada en `vector` en el parámetro formal `v` (exactamente igual que si se hubiese realizado la asignación `v = vector`).

¿Y qué sucede con el contador de referencias de ese objeto? Veamos:

In [16]:
def to_float(v):
    print(f'Dentro de to_float :: id(v) = {id(v)}')
    print(f'Dentro de to_float :: refcount(v) = {getrefcount(v)}')
    for i in range(len(v)):
        v[i] = float(v[i])

vector = [1, 7, 0, 3, 5]
print(f'Antes de la llamada :: vector = {vector}')
print(f'Antes de la llamada :: id(vector) = {id(vector)}')
print(f'Antes de la llamada :: refcount(vector) = {getrefcount(vector)}')
to_float(vector)
print(f'Después de la llamada :: vector = {vector}')
print(f'Después de la llamada :: refcount(vector) = {getrefcount(vector)}')

Antes de la llamada :: vector = [1, 7, 0, 3, 5]
Antes de la llamada :: id(vector) = 140151334332544
Antes de la llamada :: refcount(vector) = 2
Dentro de to_float :: id(v) = 140151334332544
Dentro de to_float :: refcount(v) = 4
Después de la llamada :: vector = [1.0, 7.0, 0.0, 3.0, 5.0]
Después de la llamada :: refcount(vector) = 2


Recuérdese que `getrefcount` retorna una referencia más. Obviamente, antes de llamar a la función hay una sola referencia al objeto (desde `vector`). Al llamar a la función, se crean dos nuevas referencias: una corresponde con seguridad al parámetro `v`; la otra seguramente se crea como parte del mecanismo de llamada a la función. Una vez que se retorna de la función, volvemos a tener una sola referencia (desde `vector`). Las referencias creadas al llamar a la función desaparecen al terminar de ejecutarse.

¿Pero qué sucede si al llamar a una función escribimos una expresión como parámetro?

Considerese la función `del_zeros(v)` que elimina los ceros del vector `v` y retorna el vector resultante. Sea `vector = [1, 0, 3, 0, 5]` y supongamos que hacemos la llamada: `del_zeros(vector+vector)`. Python simplemente crea un objeto nuevo automáticamente para almacenar el resultado de la expresión `vector+vector` (una lista creada creada al vuelo, recordemos la naturaleza dinámica de Python), de modo que lo que se pasa como parámetro es la referencia a dicho objeto.

In [17]:
def del_zeros(v):
    print(f'Dentro de del_zeros :: id(v) = {id(v)}')
    print(f'Dentro de del_zeros :: refcount(v) = {getrefcount(v)}')
    i = 0
    while i<len(v):
        if v[i] == 0:
            del v[i]
        else:
            i += 1
    return v

vector = [1, 0, 3, 0, 5]

print(f'Antes: refcount(vector[{id(vector)}]) = {getrefcount(vector)}')
vector_sin_ceros = del_zeros(vector+vector)
print(f'Después: refcount(vector[{id(vector)}]) = {getrefcount(vector)}')
print(f'Después: refcount(vector_sin_ceros[{id(vector_sin_ceros)}]) = {getrefcount(vector_sin_ceros)}')
print(f'vector :: {vector}')
print(f'vector_sin_ceros :: {vector_sin_ceros}')

Antes: refcount(vector[140151325379136]) = 2
Dentro de del_zeros :: id(v) = 140151325379200
Dentro de del_zeros :: refcount(v) = 3
Después: refcount(vector[140151325379136]) = 2
Después: refcount(vector_sin_ceros[140151325379200]) = 2
vector :: [1, 0, 3, 0, 5]
vector_sin_ceros :: [1, 3, 5, 1, 3, 5]


Podemos ver que `v` (dentro de la función) y `vector_sin_ceros` hacen referencia a un mismo objeto, que es diferente del referenciado por `vector`. Ese nuevo objeto es el resultado de evaluar la expresión `vector+vector`.

## Copia superficial vs copia en profundidad.

La manipulación de objetos mediante referencias es un mecanismo sencillo y eficiente. Sin embargo, en ocasiones puede tener efectos inesperados para quienes se inician en Python; o incluso para programadores experimentados, pero habituados a los lenguajes clásicos.

Por ejemplo, si se desea copiar un objetos mutable, como una lista, instrucciones que son comunes en otros lenguajes no funcionan en Python. La secuencia siguiente no consigue el objetivo deseado:

    l1 = [1, 2, 3]
    l2 = l1

puesto que con `l2 = l1` tan sólo estamos copiando una referencia, y como resultado `l2` y `l1` harán referencia al mismo objeto. Una primera solución en Python es emplear una expresión de indexación que abarque todos los elementos de la lista original, lo cual genera una copia de la lista:

    l2 = l1[:]

Esto es lo que conocemos como **copia superficial** (*shallow copy*), de modo que podemos añadir, eliminar o reemplazar elementos en `l2` sin afectar a `l1`. Este mecanismo es suficiente cuando todos los elementos de la lista son inmutables. Pero ¿qué sucede si algunos de los elementos de `l1` son mutables (por ejemplo, listas)? Veamos un ejemplo:

In [19]:
l1 = [1, 2, ['a', 'b', 'c']]
l2 = l1[:]

l2.append(666)
l2[2][0] = 'ZZZ'

print(f'l1 :: {l1}')
print(f'l2 :: {l2}')

l1 :: [1, 2, ['ZZZ', 'b', 'c']]
l2 :: [1, 2, ['ZZZ', 'b', 'c'], 666]


No debería ser una sorpresa el resultado observado, ya que los elementos de una lista son referencias a objetos. En este caso, `l1[2]` y `l2[2]` hacen referencia a la misma lista y por ello el cambio del primer elemento de la lista que hacemos desde `l2[2]` tendrá efecto sobre `l1[2]` (sencillamente porque ambos hacen referencia al mismo objeto).

Ahora bien, ¿existe algún mecanismo que permita hacer una copia completa de un objeto sin que original y copia compartan referencias a los mismos objetos? Esta copia completa e independiente del objeto original se conoce como **copia en profundidad** (*deep copy*). Veamos una primera forma de conseguirlo:

In [20]:
l1 = [1, 2, ['a', 'b', 'c']]
l2 = eval(repr(l1))   # <--- ¡TRUCO!

l2.append(666)
l2[2][0] = 'ZZZ'

print(f'l1 :: {l1}')
print(f'l2 :: {l2}')

l1 :: [1, 2, ['a', 'b', 'c']]
l2 :: [1, 2, ['ZZZ', 'b', 'c'], 666]


El *truco* se basa en esta información que proporciona `help(repr)`:

    repr(obj, /)
        Return the canonical string representation of the object.
    
        For many object types, including most builtins, eval(repr(obj)) == obj.

Nótese que no se garantiza que esto se verifique para *todos* los tipos de objetos. En el caso de las clases que definamos nosotros (algo que veremos más adelante en el curso), será responsabilidad nuestra diseñar un método `repr` que cumpla esa propiedad. Además, habrá objetos de gran complejidad para los cuales el método `repr` ya no podrá generar una representación canónica que permita recuperar el objeto original llamando a `eval`. Así pues, este truco nos valdrá a veces pero no es una solución general al problema.

Afortunadamente, existe una **solución general**: emplear las funciones definidas en el [módulo `copy` de la biblioteca estándar de Python](https://docs.python.org/3/library/copy.html).

In [21]:
import copy

l1 = [1, 2, ['a', 'b', 'c']]
l2 = copy.deepcopy(l1)

l2.append(666)
l2[2][0] = 'ZZZ'

print(f'l1 :: {l1}')
print(f'l2 :: {l2}')

l1 :: [1, 2, ['a', 'b', 'c']]
l2 :: [1, 2, ['ZZZ', 'b', 'c'], 666]


El módulo `copy` ofrece dos funciones: `copy.deepcopy`, que realiza una copia *en profundidad* del objeto que se le pasa como parámetro; y `copy.copy`, que realiza una copia *superficial*. Téngase en cuenta que, si bien es posible realizar copias superficiales de listas mediante la expresión de indexación `[:]`, y de diccionarios mediante su método `dict.copy`, necesitaremos `copy.copy` para realizar copias superficiales de objetos de otras clases.

Quizá en este punto alguien se pregunte qué sucede con las cadenas y las tuplas. Bien, recuérdese que en ambos casos se trata de objetos inmutables, por lo que no tiene ningún sentido realizar copias superficiales de ellos: ¿para qué vamos a hacer una copia superficial, si no es posible modificar el original? Sin embargo, sí que puede tener sentido realizar una copia en profundidad de las tuplas, dado que éstas pueden contener referencias a objetos mutables a cualquier nivel de profundidad.

In [22]:
t1 = (1, 2, ['a', 'b', 'c']) # <-- el tercer elemento de la tupla hace referencia a una lista (mutable)
t2 = t1[:] # copia superficial

t2[2][0] = 'ZZZ'

print(f't1 :: {t1}')
print(f't2 :: {t2}\n')

t1 = (1, 2, ['a', 'b', 'c'])
t2 = copy.deepcopy(t1) # copia en profundidad

t2[2][0] = 'ZZZ'

print(f't1 :: {t1}')
print(f't2 :: {t2}')

t1 :: (1, 2, ['ZZZ', 'b', 'c'])
t2 :: (1, 2, ['ZZZ', 'b', 'c'])

t1 :: (1, 2, ['a', 'b', 'c'])
t2 :: (1, 2, ['ZZZ', 'b', 'c'])
